In [2]:
import langchain
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()  # This loads the .env variables into the environment

secret_key = os.getenv('SQL_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [4]:
from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase.from_uri("sqlite:///afl.db")
llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [5]:
db_chain.run("In how many games did Christian Petracca have more disposals than Clayton Oliver in 2022?")



> Entering new SQLDatabaseChain chain...
In how many games did Christian Petracca have more disposals than Clayton Oliver in 2022?
SQLQuery:SELECT COUNT(*) FROM player_stats WHERE year = 2022 AND player_name = 'Christian Petracca' AND disp > (SELECT disp FROM player_stats WHERE year = 2022 AND player_name = 'Clayton Oliver');
SQLResult: [(5,)]
Answer:Christian Petracca had more disposals than Clayton Oliver in 5 games in 2022.
> Finished chain.


'Christian Petracca had more disposals than Clayton Oliver in 5 games in 2022.'

In [7]:
db_chain.run("How many times has Harris Andrews had the most One Percenters in a given match?")
# SELECT COUNT(*) FROM player_stats WHERE one_percenters = (SELECT MAX(one_percenters) FROM player_stats WHERE player_name = 'Harris Andrews');
# SELECT COUNT(*) FROM player_stats WHERE player_name = 'Harris Andrews' AND one_percenters = (SELECT MAX(one_percenters) FROM player_stats);
# BOTH WRONG. I THINK IT NEEDS TO BE GIVEN A MATCH_ID IN THE DATABASE.



> Entering new SQLDatabaseChain chain...
How many times has Harris Andrews had the most One Percenters in a given match?
SQLQuery:SELECT COUNT(*) FROM player_stats WHERE player_name = 'Harris Andrews' AND one_percenters = (SELECT MAX(one_percenters) FROM player_stats);
SQLResult: [(1,)]
Answer:Harris Andrews has had the most One Percenters in a given match 1 time.
> Finished chain.


'Harris Andrews has had the most One Percenters in a given match 1 time.'

In [11]:
db_chain.run("What is the lowest number of disposals for which a player has received 3 brownlow votes? Who was it and what year and round?")
# Correct!



> Entering new SQLDatabaseChain chain...
What is the lowest number of disposals for which a player has received 3 brownlow votes? Who was it and what year and round?
SQLQuery:SELECT player_name, year, round, MIN(disp) FROM player_stats WHERE brownlow = 3;
SQLResult: [('Max King', 2022, 10, 7)]
Answer:The lowest number of disposals for which a player has received 3 brownlow votes was 7, by Max King in round 10 of 2022.
> Finished chain.


'The lowest number of disposals for which a player has received 3 brownlow votes was 7, by Max King in round 10 of 2022.'

In [12]:
db_chain.run("How many games have Christian Petracca and Clayton Oliver played together?")



> Entering new SQLDatabaseChain chain...
How many games have Christian Petracca and Clayton Oliver played together?
SQLQuery:SELECT COUNT(*) FROM player_stats WHERE player_name IN ('Christian Petracca', 'Clayton Oliver') GROUP BY round, year HAVING COUNT(*) > 1;
SQLResult: [(2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,

'Christian Petracca and Clayton Oliver have played together in 92 games.'

In [13]:
db_chain.run("What are the top 5 games by disposals but 0 brownlow votes? Who, when, and how many disposals?")



> Entering new SQLDatabaseChain chain...
What are the top 5 games by disposals but 0 brownlow votes? Who, when, and how many disposals?
SQLQuery:SELECT player_name, year, round, disp FROM player_stats WHERE brownlow = 0 ORDER BY disp DESC LIMIT 5;
SQLResult: [('Matt Crouch', 2019, 23, 47), ('Tom Mitchell', 2018, 9, 46), ('Lachie Neale', 2021, 'QF', 46), ('Matt Priddis', 2008, 14, 45), ('Matthew Boyd', 2011, 5, 45)]
Answer:The top 5 games by disposals but 0 brownlow votes are Matt Crouch in 2019 Round 23 with 47 disposals, Tom Mitchell in 2018 Round 9 with 46 disposals, Lachie Neale in 2021 Qualifying Final with 46 disposals, Matt Priddis in 2008 Round 14 with 45 disposals, and Matthew Boyd in 2011 Round 5 with 45 disposals.
> Finished chain.


'The top 5 games by disposals but 0 brownlow votes are Matt Crouch in 2019 Round 23 with 47 disposals, Tom Mitchell in 2018 Round 9 with 46 disposals, Lachie Neale in 2021 Qualifying Final with 46 disposals, Matt Priddis in 2008 Round 14 with 45 disposals, and Matthew Boyd in 2011 Round 5 with 45 disposals.'

In [16]:
db_chain.run("How many times has a player kicked 9 goals and at least 1 behind in a game? Tell me about those games, player, year, round etc.")



> Entering new SQLDatabaseChain chain...
How many times has a player kicked 9 goals and at least 1 behind in a game? Tell me about those games, player, year, round etc.
SQLQuery:SELECT player_name, year, round, team, opponent FROM player_stats WHERE goals = 9 AND behinds >= 1 LIMIT 5;
SQLResult: [('Gary Ablett', 1996, 1, 'GE', 'ME'), ('Stewart Loewe', 1996, 2, 'SK', 'GE'), ('Gary Ablett', 1996, 9, 'GE', 'CW'), ('Tony Lockett', 1996, 10, 'SY', 'FI'), ('Jason Dunstall', 1996, 11, 'HW', 'AD')]
Answer:There have been 5 games in which a player kicked 9 goals and at least 1 behind. Those players were Gary Ablett (1996, Round 1, GE vs ME), Stewart Loewe (1996, Round 2, SK vs GE), Gary Ablett (1996, Round 9, GE vs CW), Tony Lockett (1996, Round 10, SY vs FI), and Jason Dunstall (1996, Round 11, HW vs AD).
> Finished chain.


'There have been 5 games in which a player kicked 9 goals and at least 1 behind. Those players were Gary Ablett (1996, Round 1, GE vs ME), Stewart Loewe (1996, Round 2, SK vs GE), Gary Ablett (1996, Round 9, GE vs CW), Tony Lockett (1996, Round 10, SY vs FI), and Jason Dunstall (1996, Round 11, HW vs AD).'